In [1]:
import serial

import time
import csv
import matplotlib
matplotlib.use("tkAgg")
import matplotlib.pyplot as plt
import numpy as np

import re
import pandas as pd
import dask.dataframe as dd
import streamz.dataframe as sdf
# from streamz.dataframe import StreamingDataFrame

# import threading, queue
import multiprocess as mp
# from multiprocessing import Process, Pipe
import signal
import os
from subprocess import check_output

from scipy.ndimage import gaussian_filter1d

In [2]:
portAddr = "/dev/cu.usbmodem14101"
baud = 115200

In [3]:
ser = serial.Serial(
    port=portAddr,
    baudrate=baud,
    bytesize=8,
    timeout=2,
    stopbits=serial.STOPBITS_ONE
)
ser.flushInput()



In [4]:
# plot_window = 20
# y_var = np.array(np.zeros([plot_window]))

# plt.ion()
# fig, ax = plt.subplots()
# line, = ax.plot(y_var)


In [22]:
def process_serial_out(ascii_string):
    ascii_string += ','
    findall = re.findall(r'(?P<var>.*?)=(?P<out>.*?),', ascii_string)
    extracted = {k.strip().lower(): float(v.strip()) for k,v in findall}
    return extracted

In [23]:

def arduino_worker(queue):
    print("worker starting...")
    while True:
        try:
            ser_bytes = ser.readline()
            try:
                print("got", ser_bytes)
                decoded_bytes = ser_bytes.decode('ascii')
                try:
                    row = process_serial_out(decoded_bytes)
                except:
                    continue
                print(decoded_bytes)
                queue.put(row)
            except:
                print_exc()
                print("exception reading one line from arduino")
                continue
        except:
            print_exc()
            print("Keyboard Interrupt")
            break

In [7]:
import random

def stride_boundary(row, this_stride):
#     just a dummy code. We need to implement the logic.
    if (bool(random.getrandbits(1))):
        return True
    else:
        return False

In [8]:
## NOTE!! MKIMMINS: this now has accel z smooth in each stride object.
class Stride:
    def __init__(self, df_walk_data, start_idx, stop_idx, accel_z_smooth):
        self.start_idx = start_idx
        self.stop_idx = stop_idx
        self.df_walk_data = df_walk_data
        self.accel_z_smooth = accel_z_smooth

        total_disp, integral1, integral2 = get_displacement_of_walk_segment (df_walk_data, start_idx, stop_idx)
        self.horizontal_length = abs(total_disp)
        self.time_duration = (df_walk_data.time[self.stop_idx] - df_walk_data.time[self.start_idx])/1000
        self.speed = self.horizontal_length/self.time_duration #m/s


def get_displacement_of_walk_segment(df_data, start_idx, end_idx):
    # first integration
    integral_0 = df_data.accel_z[start_idx:end_idx]

    # first integral
    cumsum = 0
    integral_1 = []
    for idx, elem in enumerate(integral_0):
        delta_time = (df_data.time[start_idx+idx+1] - df_data.time[start_idx+idx] ) / 1000
        cumsum += elem * delta_time
        integral_1.append(cumsum)
    
    # second integral
    cumsum = 0
    integral_2 = []
    for idx, elem in enumerate(integral_1):
        delta_time = (df_data.time[start_idx+idx+1] - df_data.time[start_idx+idx] ) / 1000
        cumsum += elem * delta_time
        integral_2.append(cumsum)
    
    # total_displacement, first_integral, second_integral
    return cumsum, integral_1, integral_2

# TODO Mkimmins : segment increased to 500!
# segment_length = 100
segment_length = 200
temp_segment_compute_limit_start = 0
temp_segment_compute_limit_end = 400
# temp_segment_compute_limit_end = len(readings)
total_walk = []
# strides = []

# Algorithm Constants
start_quiet_noise_idx = 0
end_quiet_noise_idx = 100 # 1 second sigma calibration
STRIDE_TIMEOUT = 500
num_of_stddev = 3
gaussian_filter_sigma = 5


In [9]:

def process_calculate_strides(total_data, total_strides):
    print("starting calculate_strides...")
    while True:
        try:
            walk_segment = total_data.get()
            print ("processing one segment chunk")
#             print("the segment: ", walk_segment)
            for k in walk_segment:total_walk.append(k) 
            df_walk = pd.DataFrame(total_walk)
            df_walk.time = df_walk.time - df_walk.time[0] # reset time from start of stream
            # Filter out accel_z
            accel_z_smooth = gaussian_filter1d(df_walk.accel_z, gaussian_filter_sigma)
            # Define Baseline Noise

            accel_z_std = np.std(df_walk.accel_z[start_quiet_noise_idx:end_quiet_noise_idx], axis=0)
            accel_z_mean = np.mean(df_walk.accel_z[start_quiet_noise_idx:end_quiet_noise_idx])
            upper_limit = accel_z_mean + num_of_stddev * accel_z_std
            lower_limit = accel_z_mean - num_of_stddev * accel_z_std

            # Determining Stride
            # Stride Class with auto-compute all chracterisitcs
            stride_under = False
            stride_over = False
            start_stride_idx, end_stride_idx = None, None
            # Determining Stride
            # Stride Class with auto-compute all chracterisitcs
            stride_under = False
            stride_over = False
            start_stride_idx, end_stride_idx = None, None

            for idx, data in enumerate(accel_z_smooth):
                # timeout
                if start_stride_idx and idx-start_stride_idx > STRIDE_TIMEOUT:
                    start_stride_idx = None
                    stride_under = False
                    stride_over = False
                if (data < lower_limit) and (not stride_under and not stride_over):
                    # Heal off the ground
                    stride_under = True
                    start_stride_idx = idx
                elif (data > upper_limit) and (stride_under and not stride_over):
                    stride_under = False
                    stride_over = True
                elif (data < upper_limit) and (not stride_under and stride_over):
                    stride_under = False
                    stride_over = False   
                    end_stride_idx = idx
                    curr_stride = Stride (df_walk, start_stride_idx, end_stride_idx, accel_z_smooth)
                    print("end of reading for one stride.")
                    total_strides.put(curr_stride)
        except:
            print_exc()
            print("exception in stride processor worker")


In [10]:


#worker to process one stride boundary, sort data into buckets
def stride_processor_worker(total_data, raw_stream):
    this_segment = []
    print("stride_processor starting...")
    while True:
        #Infinitely 1. appends to this_segment, then when boundary is met,
        #appends that stride to total_data, reinitializes this_segment
        #and loops forever.
        try:
            row = raw_stream.get()
            print ("decoded: ", row)
            if len(this_segment) > segment_length:
                #put away the current stride in total data as one stride
                total_data.put(this_segment)
                print("finished writing this one stride, size: ", len(this_segment))
                #reinitialize the stride queue
                this_segment = []
            else:
                this_segment.append(row)
        except:
            print_exc()
            print("exception in stride processor worker")
            
    

In [11]:
# overall structure:
# total_data = [stride1, stride2,,,,,,stride500]
# stride1 = [{r1}, {r2}, {r3}]

In [12]:
######### start 'main' ########

In [48]:
pool = mp.Pool()
manager = mp.Manager()
usb_reading_queue = mp.SimpleQueue() # This queue holds all individual readings that are parsed.
total_data = mp.SimpleQueue() #This queue holds all chunked readings that are emptied from reading queue
total_strides = mp.SimpleQueue() #This queue drains total_data and chunks into strides with Prathik's computation



In [49]:
#  set up worker threads

In [50]:
stride_process = mp.Process(target=stride_processor_worker, args=[total_data, usb_reading_queue])


In [51]:
arduino_proc = mp.Process(target=arduino_worker, args=[usb_reading_queue])


In [52]:
strides_compute_proc = mp.Process(target=process_calculate_strides, args=[total_data, total_strides])

In [53]:
arduino_proc.start()

worker starting...
got b'time=1535754,accel_x= -0.26,accel_y= 0.06,accel_z= 0.19,gyro_x= 0.02,gyro_y= 0.07,gyro_z= -0.02,mag_x= -36.69,mag_y= -6.75,mag_z= -37.06,orientation_x= 247.37,orientation_y= 83.69,orientation_z= -175.87\r\n'
time=1535754,accel_x= -0.26,accel_y= 0.06,accel_z= 0.19,gyro_x= 0.02,gyro_y= 0.07,gyro_z= -0.02,mag_x= -36.69,mag_y= -6.75,mag_z= -37.06,orientation_x= 247.37,orientation_y= 83.69,orientation_z= -175.87

got b'time=1535766,accel_x= -0.40,accel_y= 0.05,accel_z= 0.45,gyro_x= 0.01,gyro_y= 0.06,gyro_z= -0.01,mag_x= -36.69,mag_y= -6.75,mag_z= -37.06,orientation_x= 247.37,orientation_y= 83.69,orientation_z= -175.87\r\n'
time=1535766,accel_x= -0.40,accel_y= 0.05,accel_z= 0.45,gyro_x= 0.01,gyro_y= 0.06,gyro_z= -0.01,mag_x= -36.69,mag_y= -6.75,mag_z= -37.06,orientation_x= 247.37,orientation_y= 83.69,orientation_z= -175.87

got b'time=1535777,accel_x= -0.34,accel_y= 0.05,accel_z= 0.34,gyro_x= 0.01,gyro_y= 0.02,gyro_z= -0.00,mag_x= -36.69,mag_y= -6.75,mag_z= -37.06,or

got b'time=1535963,accel_x= -3.35,accel_y= 2.14,accel_z= 3.42,gyro_x= -0.28,gyro_y= -0.26,gyro_z= 0.29,mag_x= -37.00,mag_y= -6.75,mag_z= -36.69,orientation_x= 247.56,orientation_y= 83.62,orientation_z= -176.50\r\n'
time=1535963,accel_x= -3.35,accel_y= 2.14,accel_z= 3.42,gyro_x= -0.28,gyro_y= -0.26,gyro_z= 0.29,mag_x= -37.00,mag_y= -6.75,mag_z= -36.69,orientation_x= 247.56,orientation_y= 83.62,orientation_z= -176.50

got b'time=1535976,accel_x= -3.83,accel_y= 2.09,accel_z= 3.79,gyro_x= -0.32,gyro_y= -0.36,gyro_z= 0.36,mag_x= -37.00,mag_y= -6.75,mag_z= -36.69,orientation_x= 247.56,orientation_y= 83.56,orientation_z= -176.69\r\n'
time=1535976,accel_x= -3.83,accel_y= 2.09,accel_z= 3.79,gyro_x= -0.32,gyro_y= -0.36,gyro_z= 0.36,mag_x= -37.00,mag_y= -6.75,mag_z= -36.69,orientation_x= 247.56,orientation_y= 83.56,orientation_z= -176.69

got b'time=1535986,accel_x= -3.04,accel_y= 1.69,accel_z= 3.13,gyro_x= -0.38,gyro_y= -0.50,gyro_z= 0.45,mag_x= -37.00,mag_y= -6.06,mag_z= -36.69,orientation_x= 2


got b'time=1536188,accel_x= 1.29,accel_y= -2.46,accel_z= -0.75,gyro_x= -0.00,gyro_y= -0.20,gyro_z= 0.18,mag_x= -44.69,mag_y= -2.50,mag_z= -30.56,orientation_x= 242.56,orientation_y= 74.44,orientation_z= 173.56\r\n'
time=1536188,accel_x= 1.29,accel_y= -2.46,accel_z= -0.75,gyro_x= -0.00,gyro_y= -0.20,gyro_z= 0.18,mag_x= -44.69,mag_y= -2.50,mag_z= -30.56,orientation_x= 242.56,orientation_y= 74.44,orientation_z= 173.56

got b'time=1536199,accel_x= 0.44,accel_y= -2.25,accel_z= 0.71,gyro_x= -0.35,gyro_y= -0.10,gyro_z= 0.17,mag_x= -44.69,mag_y= -2.50,mag_z= -30.56,orientation_x= 242.75,orientation_y= 74.37,orientation_z= 173.44\r\n'
time=1536199,accel_x= 0.44,accel_y= -2.25,accel_z= 0.71,gyro_x= -0.35,gyro_y= -0.10,gyro_z= 0.17,mag_x= -44.69,mag_y= -2.50,mag_z= -30.56,orientation_x= 242.75,orientation_y= 74.37,orientation_z= 173.44

got b'time=1536212,accel_x= 0.29,accel_y= -1.96,accel_z= -0.25,gyro_x= -0.54,gyro_y= 0.04,gyro_z= 0.07,mag_x= -44.69,mag_y= -2.50,mag_z= -30.56,orientation_x= 24


got b'time=1536416,accel_x= 0.02,accel_y= 0.08,accel_z= -0.05,gyro_x= 0.14,gyro_y= 0.10,gyro_z= 0.04,mag_x= -45.38,mag_y= -0.25,mag_z= -31.50,orientation_x= 241.81,orientation_y= 74.75,orientation_z= 171.75\r\n'
time=1536416,accel_x= 0.02,accel_y= 0.08,accel_z= -0.05,gyro_x= 0.14,gyro_y= 0.10,gyro_z= 0.04,mag_x= -45.38,mag_y= -0.25,mag_z= -31.50,orientation_x= 241.81,orientation_y= 74.75,orientation_z= 171.75

got b'time=1536427,accel_x= 0.02,accel_y= -0.14,accel_z= 0.02,gyro_x= -0.22,gyro_y= 0.02,gyro_z= 0.05,mag_x= -45.38,mag_y= -0.25,mag_z= -31.50,orientation_x= 241.81,orientation_y= 74.75,orientation_z= 171.69\r\n'
time=1536427,accel_x= 0.02,accel_y= -0.14,accel_z= 0.02,gyro_x= -0.22,gyro_y= 0.02,gyro_z= 0.05,mag_x= -45.38,mag_y= -0.25,mag_z= -31.50,orientation_x= 241.81,orientation_y= 74.75,orientation_z= 171.69

got b'time=1536437,accel_x= 0.02,accel_y= -0.30,accel_z= 0.09,gyro_x= -0.22,gyro_y= 0.02,gyro_z= 0.05,mag_x= -45.00,mag_y= -2.87,mag_z= -31.50,orientation_x= 241.94,orie


got b'time=1536638,accel_x= 0.95,accel_y= -0.51,accel_z= -1.28,gyro_x= 0.13,gyro_y= 0.50,gyro_z= -0.38,mag_x= -45.00,mag_y= -2.56,mag_z= -31.87,orientation_x= 244.25,orientation_y= 75.81,orientation_z= 176.56\r\n'
time=1536638,accel_x= 0.95,accel_y= -0.51,accel_z= -1.28,gyro_x= 0.13,gyro_y= 0.50,gyro_z= -0.38,mag_x= -45.00,mag_y= -2.56,mag_z= -31.87,orientation_x= 244.25,orientation_y= 75.81,orientation_z= 176.56

got b'time=1536649,accel_x= 0.28,accel_y= 0.03,accel_z= -0.28,gyro_x= 0.10,gyro_y= 0.55,gyro_z= -0.40,mag_x= -45.00,mag_y= -2.56,mag_z= -31.87,orientation_x= 244.37,orientation_y= 76.12,orientation_z= 177.25\r\n'
time=1536649,accel_x= 0.28,accel_y= 0.03,accel_z= -0.28,gyro_x= 0.10,gyro_y= 0.55,gyro_z= -0.40,mag_x= -45.00,mag_y= -2.56,mag_z= -31.87,orientation_x= 244.37,orientation_y= 76.12,orientation_z= 177.25

got b'time=1536661,accel_x= -0.07,accel_y= 0.22,accel_z= 0.03,gyro_x= 0.11,gyro_y= 0.60,gyro_z= -0.42,mag_x= -45.00,mag_y= -2.56,mag_z= -31.87,orientation_x= 244.69,

...

got b'time=1552813,accel_x= -0.17,accel_y= -0.07,accel_z= 0.21,gyro_x= -0.16,gyro_y= -0.14,gyro_z= 0.09,mag_x= -34.69,mag_y= -6.37,mag_z= -38.50,orientation_x= 244.94,orientation_y= 83.50,orientation_z= -176.62\r\n'
time=1552813,accel_x= -0.17,accel_y= -0.07,accel_z= 0.21,gyro_x= -0.16,gyro_y= -0.14,gyro_z= 0.09,mag_x= -34.69,mag_y= -6.37,mag_z= -38.50,orientation_x= 244.94,orientation_y= 83.50,orientation_z= -176.62

got b'time=1552826,accel_x= -0.09,accel_y= -0.22,accel_z= 0.10,gyro_x= -0.15,gyro_y= -0.16,gyro_z= 0.09,mag_x= -34.69,mag_y= -6.37,mag_z= -38.50,orientation_x= 244.94,orientation_y= 83.44,orientation_z= -176.69\r\n'
time=1552826,accel_x= -0.09,accel_y= -0.22,accel_z= 0.10,gyro_x= -0.15,gyro_y= -0.16,gyro_z= 0.09,mag_x= -34.69,mag_y= -6.37,mag_z= -38.50,orientation_x= 244.94,orientation_y= 83.44,orientation_z= -176.69

got b'time=1552837,accel_x= 0.01,accel_y= -0.22,accel_z= 0.00,gyro_x= -0.13,gyro_y= -0.16,gyro_z= 0.09,mag_x= -34.69,mag_y= -6.37,mag_z= -38.50,orientation_

got b'time=1553029,accel_x= 1.32,accel_y= 0.19,accel_z= -2.17,gyro_x= 0.58,gyro_y= -0.59,gyro_z= 0.29,mag_x= -37.69,mag_y= -3.69,mag_z= -36.06,orientation_x= 242.81,orientation_y= 76.69,orientation_z= 175.50\r\n'
time=1553029,accel_x= 1.32,accel_y= 0.19,accel_z= -2.17,gyro_x= 0.58,gyro_y= -0.59,gyro_z= 0.29,mag_x= -37.69,mag_y= -3.69,mag_z= -36.06,orientation_x= 242.81,orientation_y= 76.69,orientation_z= 175.50

got b'time=1553039,accel_x= 1.45,accel_y= -0.63,accel_z= -2.47,gyro_x= 0.37,gyro_y= -0.67,gyro_z= 0.23,mag_x= -40.19,mag_y= -0.75,mag_z= -33.06,orientation_x= 242.37,orientation_y= 76.25,orientation_z= 174.87\r\n'
time=1553039,accel_x= 1.45,accel_y= -0.63,accel_z= -2.47,gyro_x= 0.37,gyro_y= -0.67,gyro_z= 0.23,mag_x= -40.19,mag_y= -0.75,mag_z= -33.06,orientation_x= 242.37,orientation_y= 76.25,orientation_z= 174.87

got b'time=1553049,accel_x= 1.45,accel_y= -0.86,accel_z= -2.10,gyro_x= 0.20,gyro_y= -0.73,gyro_z= 0.23,mag_x= -40.19,mag_y= -0.75,mag_z= -33.06,orientation_x= 242.12,


got b'time=1553252,accel_x= -0.03,accel_y= 0.01,accel_z= 0.05,gyro_x= 0.06,gyro_y= -0.04,gyro_z= 0.02,mag_x= -42.88,mag_y= -2.00,mag_z= -32.50,orientation_x= 240.19,orientation_y= 74.31,orientation_z= 172.00\r\n'
time=1553252,accel_x= -0.03,accel_y= 0.01,accel_z= 0.05,gyro_x= 0.06,gyro_y= -0.04,gyro_z= 0.02,mag_x= -42.88,mag_y= -2.00,mag_z= -32.50,orientation_x= 240.19,orientation_y= 74.31,orientation_z= 172.00

got b'time=1553265,accel_x= 0.00,accel_y= -0.16,accel_z= 0.00,gyro_x= 0.04,gyro_y= 0.02,gyro_z= 0.06,mag_x= -42.88,mag_y= -2.00,mag_z= -32.50,orientation_x= 240.06,orientation_y= 74.31,orientation_z= 171.81\r\n'
time=1553265,accel_x= 0.00,accel_y= -0.16,accel_z= 0.00,gyro_x= 0.04,gyro_y= 0.02,gyro_z= 0.06,mag_x= -42.88,mag_y= -2.00,mag_z= -32.50,orientation_x= 240.06,orientation_y= 74.31,orientation_z= 171.81

got b'time=1553278,accel_x= 0.05,accel_y= -0.05,accel_z= 0.06,gyro_x= 0.01,gyro_y= -0.02,gyro_z= 0.06,mag_x= -42.88,mag_y= -2.00,mag_z= -32.50,orientation_x= 240.06,orie


got b'time=1553471,accel_x= -0.02,accel_y= -0.23,accel_z= 0.07,gyro_x= -0.12,gyro_y= -0.04,gyro_z= -0.01,mag_x= -42.75,mag_y= -1.25,mag_z= -32.06,orientation_x= 240.62,orientation_y= 74.62,orientation_z= 170.56\r\n'
time=1553471,accel_x= -0.02,accel_y= -0.23,accel_z= 0.07,gyro_x= -0.12,gyro_y= -0.04,gyro_z= -0.01,mag_x= -42.75,mag_y= -1.25,mag_z= -32.06,orientation_x= 240.62,orientation_y= 74.62,orientation_z= 170.56

got b'time=1553481,accel_x= 0.00,accel_y= -0.18,accel_z= 0.10,gyro_x= -0.09,gyro_y= -0.09,gyro_z= -0.02,mag_x= -42.75,mag_y= -1.25,mag_z= -32.06,orientation_x= 240.75,orientation_y= 74.56,orientation_z= 170.75\r\n'
time=1553481,accel_x= 0.00,accel_y= -0.18,accel_z= 0.10,gyro_x= -0.09,gyro_y= -0.09,gyro_z= -0.02,mag_x= -42.75,mag_y= -1.25,mag_z= -32.06,orientation_x= 240.75,orientation_y= 74.56,orientation_z= 170.75

got b'time=1553491,accel_x= 0.01,accel_y= -0.12,accel_z= 0.01,gyro_x= 0.01,gyro_y= -0.08,gyro_z= -0.03,mag_x= -42.38,mag_y= -0.50,mag_z= -32.50,orientation_x


got b'time=1553686,accel_x= 0.02,accel_y= -0.06,accel_z= 0.04,gyro_x= 0.05,gyro_y= 0.02,gyro_z= 0.00,mag_x= -43.06,mag_y= -1.25,mag_z= -32.88,orientation_x= 240.00,orientation_y= 74.44,orientation_z= 170.69\r\n'
time=1553686,accel_x= 0.02,accel_y= -0.06,accel_z= 0.04,gyro_x= 0.05,gyro_y= 0.02,gyro_z= 0.00,mag_x= -43.06,mag_y= -1.25,mag_z= -32.88,orientation_x= 240.00,orientation_y= 74.44,orientation_z= 170.69

got b'time=1553696,accel_x= 0.00,accel_y= -0.07,accel_z= 0.11,gyro_x= 0.03,gyro_y= 0.02,gyro_z= 0.01,mag_x= -43.06,mag_y= -1.25,mag_z= -32.06,orientation_x= 239.94,orientation_y= 74.44,orientation_z= 170.62\r\n'
time=1553696,accel_x= 0.00,accel_y= -0.07,accel_z= 0.11,gyro_x= 0.03,gyro_y= 0.02,gyro_z= 0.01,mag_x= -43.06,mag_y= -1.25,mag_z= -32.06,orientation_x= 239.94,orientation_y= 74.44,orientation_z= 170.62

got b'time=1553707,accel_x= -0.02,accel_y= -0.11,accel_z= 0.08,gyro_x= 0.02,gyro_y= 0.01,gyro_z= 0.01,mag_x= -43.06,mag_y= -1.25,mag_z= -32.06,orientation_x= 239.94,orient

In [54]:
stride_process.start()

stride_processor starting...
decoded:  {'time': 1535754.0, 'accel_x': -0.26, 'accel_y': 0.06, 'accel_z': 0.19, 'gyro_x': 0.02, 'gyro_y': 0.07, 'gyro_z': -0.02, 'mag_x': -36.69, 'mag_y': -6.75, 'mag_z': -37.06, 'orientation_x': 247.37, 'orientation_y': 83.69}
decoded:  {'time': 1535766.0, 'accel_x': -0.4, 'accel_y': 0.05, 'accel_z': 0.45, 'gyro_x': 0.01, 'gyro_y': 0.06, 'gyro_z': -0.01, 'mag_x': -36.69, 'mag_y': -6.75, 'mag_z': -37.06, 'orientation_x': 247.37, 'orientation_y': 83.69}
decoded:  {'time': 1535777.0, 'accel_x': -0.34, 'accel_y': 0.05, 'accel_z': 0.34, 'gyro_x': 0.01, 'gyro_y': 0.02, 'gyro_z': -0.0, 'mag_x': -36.69, 'mag_y': -6.75, 'mag_z': -37.06, 'orientation_x': 247.37, 'orientation_y': 83.75}
decoded:  {'time': 1535787.0, 'accel_x': -0.23, 'accel_y': -0.01, 'accel_z': 0.29, 'gyro_x': 0.02, 'gyro_y': 0.01, 'gyro_z': 0.0, 'mag_x': -36.56, 'mag_y': -7.19, 'mag_z': -37.06, 'orientation_x': 247.37, 'orientation_y': 83.75}
decoded:  {'time': 1535797.0, 'accel_x': -0.15, 'accel

decoded:  {'time': 1536143.0, 'accel_x': 1.19, 'accel_y': -1.02, 'accel_z': -1.56, 'gyro_x': 0.51, 'gyro_y': -0.86, 'gyro_z': 0.2, 'mag_x': -43.56, 'mag_y': -1.75, 'mag_z': -32.06, 'orientation_x': 244.25, 'orientation_y': 75.81}
decoded:  {'time': 1536155.0, 'accel_x': 1.16, 'accel_y': -0.62, 'accel_z': -1.47, 'gyro_x': 0.57, 'gyro_y': -0.69, 'gyro_z': 0.11, 'mag_x': -43.56, 'mag_y': -1.75, 'mag_z': -32.06, 'orientation_x': 243.75, 'orientation_y': 75.44}
decoded:  {'time': 1536167.0, 'accel_x': 0.21, 'accel_y': 1.53, 'accel_z': 2.66, 'gyro_x': 0.42, 'gyro_y': -0.52, 'gyro_z': 0.07, 'mag_x': -43.56, 'mag_y': -1.75, 'mag_z': -32.06, 'orientation_x': 243.37, 'orientation_y': 75.12}
decoded:  {'time': 1536178.0, 'accel_x': 2.24, 'accel_y': -0.67, 'accel_z': -4.51, 'gyro_x': 0.26, 'gyro_y': -0.44, 'gyro_z': 0.05, 'mag_x': -43.56, 'mag_y': -1.75, 'mag_z': -32.06, 'orientation_x': 242.87, 'orientation_y': 74.81}
decoded:  {'time': 1536188.0, 'accel_x': 1.29, 'accel_y': -2.46, 'accel_z': -0.

decoded:  {'time': 1536538.0, 'accel_x': 0.15, 'accel_y': -0.33, 'accel_z': -0.13, 'gyro_x': 0.0, 'gyro_y': 0.04, 'gyro_z': -0.13, 'mag_x': -45.25, 'mag_y': -2.25, 'mag_z': -30.69, 'orientation_x': 243.31, 'orientation_y': 75.06}
decoded:  {'time': 1536549.0, 'accel_x': 0.3, 'accel_y': -0.06, 'accel_z': -0.42, 'gyro_x': 0.09, 'gyro_y': 0.09, 'gyro_z': -0.15, 'mag_x': -45.25, 'mag_y': -2.25, 'mag_z': -30.69, 'orientation_x': 243.37, 'orientation_y': 75.12}
decoded:  {'time': 1536561.0, 'accel_x': 0.46, 'accel_y': -0.06, 'accel_z': -0.46, 'gyro_x': 0.02, 'gyro_y': 0.09, 'gyro_z': -0.16, 'mag_x': -45.25, 'mag_y': -2.25, 'mag_z': -30.69, 'orientation_x': 243.44, 'orientation_y': 75.19}
decoded:  {'time': 1536574.0, 'accel_x': 0.85, 'accel_y': -0.24, 'accel_z': -1.5, 'gyro_x': -0.09, 'gyro_y': 0.07, 'gyro_z': -0.15, 'mag_x': -45.25, 'mag_y': -2.25, 'mag_z': -30.69, 'orientation_x': 243.62, 'orientation_y': 75.19}
decoded:  {'time': 1536584.0, 'accel_x': 1.06, 'accel_y': -0.83, 'accel_z': -1

decoded:  {'time': 1536928.0, 'accel_x': -0.14, 'accel_y': 0.01, 'accel_z': 0.16, 'gyro_x': 0.05, 'gyro_y': 0.28, 'gyro_z': -0.13, 'mag_x': -40.19, 'mag_y': -5.19, 'mag_z': -34.25, 'orientation_x': 246.62, 'orientation_y': 81.94}
decoded:  {'time': 1536938.0, 'accel_x': -0.29, 'accel_y': 0.17, 'accel_z': 0.33, 'gyro_x': 0.02, 'gyro_y': 0.3, 'gyro_z': -0.13, 'mag_x': -38.38, 'mag_y': -5.56, 'mag_z': -35.75, 'orientation_x': 246.62, 'orientation_y': 82.06}
decoded:  {'time': 1536949.0, 'accel_x': -0.34, 'accel_y': 0.19, 'accel_z': 0.4, 'gyro_x': -0.0, 'gyro_y': 0.3, 'gyro_z': -0.12, 'mag_x': -38.38, 'mag_y': -5.56, 'mag_z': -35.75, 'orientation_x': 246.69, 'orientation_y': 82.19}
decoded:  {'time': 1536962.0, 'accel_x': -0.32, 'accel_y': 0.15, 'accel_z': 0.34, 'gyro_x': -0.01, 'gyro_y': 0.29, 'gyro_z': -0.11, 'mag_x': -38.38, 'mag_y': -5.56, 'mag_z': -35.75, 'orientation_x': 246.81, 'orientation_y': 82.44}
decoded:  {'time': 1536975.0, 'accel_x': -0.11, 'accel_y': -0.01, 'accel_z': 0.21,

decoded:  {'time': 1537318.0, 'accel_x': -2.01, 'accel_y': 1.31, 'accel_z': 2.05, 'gyro_x': -0.03, 'gyro_y': -0.43, 'gyro_z': 0.35, 'mag_x': -37.5, 'mag_y': -5.87, 'mag_z': -35.0, 'orientation_x': 246.94, 'orientation_y': 82.25}
decoded:  {'time': 1537328.0, 'accel_x': -2.18, 'accel_y': 1.33, 'accel_z': 2.43, 'gyro_x': -0.04, 'gyro_y': -0.45, 'gyro_z': 0.4, 'mag_x': -37.5, 'mag_y': -5.87, 'mag_z': -35.0, 'orientation_x': 246.56, 'orientation_y': 81.87}
decoded:  {'time': 1537338.0, 'accel_x': -1.91, 'accel_y': 0.81, 'accel_z': 2.02, 'gyro_x': -0.07, 'gyro_y': -0.56, 'gyro_z': 0.46, 'mag_x': -38.19, 'mag_y': -5.19, 'mag_z': -34.69, 'orientation_x': 246.37, 'orientation_y': 81.62}
decoded:  {'time': 1537349.0, 'accel_x': -1.45, 'accel_y': 0.27, 'accel_z': 1.4, 'gyro_x': -0.14, 'gyro_y': -0.74, 'gyro_z': 0.53, 'mag_x': -38.19, 'mag_y': -5.19, 'mag_z': -34.69, 'orientation_x': 246.19, 'orientation_y': 81.31}
decoded:  {'time': 1537362.0, 'accel_x': -0.38, 'accel_y': -0.56, 'accel_z': 0.53,

...

decoded:  {'time': 1551904.0, 'accel_x': -0.1, 'accel_y': -0.01, 'accel_z': 0.2, 'gyro_x': 0.1, 'gyro_y': 0.21, 'gyro_z': -0.09, 'mag_x': -35.06, 'mag_y': -5.87, 'mag_z': -37.56, 'orientation_x': 245.31, 'orientation_y': 82.44}
decoded:  {'time': 1551917.0, 'accel_x': -0.04, 'accel_y': -0.15, 'accel_z': 0.04, 'gyro_x': 0.12, 'gyro_y': 0.23, 'gyro_z': -0.1, 'mag_x': -35.06, 'mag_y': -5.87, 'mag_z': -37.56, 'orientation_x': 245.25, 'orientation_y': 82.56}
decoded:  {'time': 1551929.0, 'accel_x': -0.02, 'accel_y': -0.08, 'accel_z': 0.22, 'gyro_x': 0.1, 'gyro_y': 0.23, 'gyro_z': -0.11, 'mag_x': -35.06, 'mag_y': -5.87, 'mag_z': -37.56, 'orientation_x': 245.25, 'orientation_y': 82.62}
decoded:  {'time': 1551939.0, 'accel_x': -0.1, 'accel_y': -0.21, 'accel_z': 0.12, 'gyro_x': 0.04, 'gyro_y': 0.2, 'gyro_z': -0.11, 'mag_x': -36.0, 'mag_y': -5.19, 'mag_z': -37.19, 'orientation_x': 245.31, 'orientation_y': 82.75}
decoded:  {'time': 1551949.0, 'accel_x': -0.12, 'accel_y': -0.15, 'accel_z': 0.26, '

decoded:  {'time': 1552462.0, 'accel_x': 0.16, 'accel_y': -0.38, 'accel_z': -0.16, 'gyro_x': -0.11, 'gyro_y': -0.13, 'gyro_z': 0.05, 'mag_x': -34.0, 'mag_y': -6.69, 'mag_z': -38.5, 'orientation_x': 244.87, 'orientation_y': 83.75}
decoded:  {'time': 1552475.0, 'accel_x': 0.4, 'accel_y': -0.44, 'accel_z': -0.45, 'gyro_x': -0.07, 'gyro_y': -0.12, 'gyro_z': 0.04, 'mag_x': -34.0, 'mag_y': -6.69, 'mag_z': -38.5, 'orientation_x': 245.0, 'orientation_y': 83.62}
decoded:  {'time': 1552485.0, 'accel_x': 0.46, 'accel_y': -0.38, 'accel_z': -0.54, 'gyro_x': -0.02, 'gyro_y': -0.12, 'gyro_z': 0.03, 'mag_x': -34.0, 'mag_y': -6.69, 'mag_z': -38.5, 'orientation_x': 245.0, 'orientation_y': 83.62}
decoded:  {'time': 1552495.0, 'accel_x': 0.46, 'accel_y': -0.29, 'accel_z': -0.58, 'gyro_x': 0.03, 'gyro_y': -0.11, 'gyro_z': 0.01, 'mag_x': -34.38, 'mag_y': -7.37, 'mag_z': -36.5, 'orientation_x': 244.94, 'orientation_y': 83.56}
decoded:  {'time': 1552508.0, 'accel_x': 0.38, 'accel_y': -0.12, 'accel_z': -0.52, 

decoded:  {'time': 1552859.0, 'accel_x': -0.85, 'accel_y': 0.47, 'accel_z': 0.78, 'gyro_x': -0.1, 'gyro_y': -0.14, 'gyro_z': 0.08, 'mag_x': -34.0, 'mag_y': -6.69, 'mag_z': -37.25, 'orientation_x': 245.06, 'orientation_y': 83.31}
decoded:  {'time': 1552871.0, 'accel_x': -1.32, 'accel_y': 0.82, 'accel_z': 1.22, 'gyro_x': -0.08, 'gyro_y': -0.1, 'gyro_z': 0.09, 'mag_x': -34.0, 'mag_y': -6.69, 'mag_z': -37.25, 'orientation_x': 245.06, 'orientation_y': 83.25}
decoded:  {'time': 1552881.0, 'accel_x': -1.84, 'accel_y': 1.19, 'accel_z': 1.69, 'gyro_x': -0.1, 'gyro_y': -0.11, 'gyro_z': 0.13, 'mag_x': -34.0, 'mag_y': -6.69, 'mag_z': -37.25, 'orientation_x': 245.06, 'orientation_y': 83.25}
decoded:  {'time': 1552891.0, 'accel_x': -2.47, 'accel_y': 1.63, 'accel_z': 2.37, 'gyro_x': -0.15, 'gyro_y': -0.14, 'gyro_z': 0.18, 'mag_x': -35.06, 'mag_y': -6.37, 'mag_z': -37.69, 'orientation_x': 245.06, 'orientation_y': 83.19}
decoded:  {'time': 1552902.0, 'accel_x': -3.52, 'accel_y': 2.13, 'accel_z': 3.58, 

decoded:  {'time': 1553252.0, 'accel_x': -0.03, 'accel_y': 0.01, 'accel_z': 0.05, 'gyro_x': 0.06, 'gyro_y': -0.04, 'gyro_z': 0.02, 'mag_x': -42.88, 'mag_y': -2.0, 'mag_z': -32.5, 'orientation_x': 240.19, 'orientation_y': 74.31}
decoded:  {'time': 1553265.0, 'accel_x': 0.0, 'accel_y': -0.16, 'accel_z': 0.0, 'gyro_x': 0.04, 'gyro_y': 0.02, 'gyro_z': 0.06, 'mag_x': -42.88, 'mag_y': -2.0, 'mag_z': -32.5, 'orientation_x': 240.06, 'orientation_y': 74.31}
decoded:  {'time': 1553278.0, 'accel_x': 0.05, 'accel_y': -0.05, 'accel_z': 0.06, 'gyro_x': 0.01, 'gyro_y': -0.02, 'gyro_z': 0.06, 'mag_x': -42.88, 'mag_y': -2.0, 'mag_z': -32.5, 'orientation_x': 240.06, 'orientation_y': 74.31}
decoded:  {'time': 1553288.0, 'accel_x': 0.02, 'accel_y': -0.2, 'accel_z': -0.09, 'gyro_x': 0.04, 'gyro_y': -0.02, 'gyro_z': 0.07, 'mag_x': -42.88, 'mag_y': -1.25, 'mag_z': -32.88, 'orientation_x': 240.0, 'orientation_y': 74.25}
decoded:  {'time': 1553298.0, 'accel_x': 0.03, 'accel_y': 0.0, 'accel_z': 0.08, 'gyro_x': 

decoded:  {'time': 1553632.0, 'accel_x': 0.0, 'accel_y': -0.08, 'accel_z': 0.04, 'gyro_x': -0.0, 'gyro_y': -0.01, 'gyro_z': 0.01, 'mag_x': -42.75, 'mag_y': -1.69, 'mag_z': -32.88, 'orientation_x': 240.25, 'orientation_y': 74.37}
decoded:  {'time': 1553642.0, 'accel_x': 0.02, 'accel_y': -0.08, 'accel_z': 0.03, 'gyro_x': 0.05, 'gyro_y': 0.01, 'gyro_z': 0.01, 'mag_x': -43.06, 'mag_y': -1.25, 'mag_z': -32.88, 'orientation_x': 240.19, 'orientation_y': 74.37}
decoded:  {'time': 1553651.0, 'accel_x': 0.04, 'accel_y': -0.06, 'accel_z': 0.04, 'gyro_x': 0.08, 'gyro_y': 0.01, 'gyro_z': 0.01, 'mag_x': -43.06, 'mag_y': -1.25, 'mag_z': -32.88, 'orientation_x': 240.12, 'orientation_y': 74.37}
decoded:  {'time': 1553663.0, 'accel_x': 0.01, 'accel_y': -0.04, 'accel_z': 0.03, 'gyro_x': 0.09, 'gyro_y': 0.02, 'gyro_z': 0.01, 'mag_x': -43.06, 'mag_y': -1.25, 'mag_z': -32.88, 'orientation_x': 240.06, 'orientation_y': 74.44}
decoded:  {'time': 1553676.0, 'accel_x': 0.02, 'accel_y': -0.03, 'accel_z': 0.04, 'g

In [55]:
strides_compute_proc.start()

starting calculate_strides...


the segment:  [{'time': 1535754.0, 'accel_x': -0.26, 'accel_y': 0.06, 'accel_z': 0.19, 'gyro_x': 0.02, 'gyro_y': 0.07, 'gyro_z': -0.02, 'mag_x': -36.69, 'mag_y': -6.75, 'mag_z': -37.06, 'orientation_x': 247.37, 'orientation_y': 83.69}, {'time': 1535766.0, 'accel_x': -0.4, 'accel_y': 0.05, 'accel_z': 0.45, 'gyro_x': 0.01, 'gyro_y': 0.06, 'gyro_z': -0.01, 'mag_x': -36.69, 'mag_y': -6.75, 'mag_z': -37.06, 'orientation_x': 247.37, 'orientation_y': 83.69}, {'time': 1535777.0, 'accel_x': -0.34, 'accel_y': 0.05, 'accel_z': 0.34, 'gyro_x': 0.01, 'gyro_y': 0.02, 'gyro_z': -0.0, 'mag_x': -36.69, 'mag_y': -6.75, 'mag_z': -37.06, 'orientation_x': 247.37, 'orientation_y': 83.75}, {'time': 1535787.0, 'accel_x': -0.23, 'accel_y': -0.01, 'accel_z': 0.29, 'gyro_x': 0.02, 'gyro_y': 0.01, 'gyro_z': 0.0, 'mag_x': -36.56, 'mag_y': -7.19, 'mag_z': -37.06, 'orientation_x': 247.37, 'orientation_y': 83.75}, {'time': 1535797.0, 'accel_x': -0.15, 'accel_y': -0.01, 'accel_z': 0.17, 'gyro_x': 0.01, 'gyro_y': -0.01

the segment:  [{'time': 1538018.0, 'accel_x': -0.04, 'accel_y': -0.01, 'accel_z': 0.1, 'gyro_x': -0.09, 'gyro_y': 0.27, 'gyro_z': -0.11, 'mag_x': -40.88, 'mag_y': -3.75, 'mag_z': -34.56, 'orientation_x': 245.94, 'orientation_y': 80.25}, {'time': 1538029.0, 'accel_x': 0.07, 'accel_y': -0.08, 'accel_z': -0.11, 'gyro_x': -0.04, 'gyro_y': 0.13, 'gyro_z': -0.13, 'mag_x': -40.88, 'mag_y': -3.75, 'mag_z': -34.56, 'orientation_x': 246.0, 'orientation_y': 80.31}, {'time': 1538039.0, 'accel_x': 0.0, 'accel_y': 0.0, 'accel_z': -0.05, 'gyro_x': 0.02, 'gyro_y': 0.19, 'gyro_z': -0.11, 'mag_x': -39.75, 'mag_y': -2.37, 'mag_z': -33.0, 'orientation_x': 246.12, 'orientation_y': 80.37}, {'time': 1538049.0, 'accel_x': -0.04, 'accel_y': -0.01, 'accel_z': -0.04, 'gyro_x': 0.1, 'gyro_y': 0.28, 'gyro_z': -0.11, 'mag_x': -39.75, 'mag_y': -2.37, 'mag_z': -33.0, 'orientation_x': 246.12, 'orientation_y': 80.5}, {'time': 1538061.0, 'accel_x': -0.07, 'accel_y': 0.05, 'accel_z': 0.04, 'gyro_x': 0.14, 'gyro_y': 0.3, 

the segment:  [{'time': 1540286.0, 'accel_x': -0.01, 'accel_y': -0.06, 'accel_z': -0.01, 'gyro_x': 0.03, 'gyro_y': -0.0, 'gyro_z': 0.01, 'mag_x': -36.06, 'mag_y': -7.37, 'mag_z': -38.25, 'orientation_x': 246.25, 'orientation_y': 83.31}, {'time': 1540297.0, 'accel_x': 0.02, 'accel_y': -0.09, 'accel_z': -0.05, 'gyro_x': -0.0, 'gyro_y': -0.04, 'gyro_z': 0.01, 'mag_x': -36.06, 'mag_y': -7.37, 'mag_z': -38.25, 'orientation_x': 246.19, 'orientation_y': 83.25}, {'time': 1540309.0, 'accel_x': 0.13, 'accel_y': -0.15, 'accel_z': -0.17, 'gyro_x': -0.04, 'gyro_y': -0.01, 'gyro_z': 0.02, 'mag_x': -36.06, 'mag_y': -7.37, 'mag_z': -38.25, 'orientation_x': 246.25, 'orientation_y': 83.25}, {'time': 1540321.0, 'accel_x': 0.15, 'accel_y': -0.19, 'accel_z': -0.18, 'gyro_x': -0.03, 'gyro_y': 0.01, 'gyro_z': 0.02, 'mag_x': -36.06, 'mag_y': -7.37, 'mag_z': -38.25, 'orientation_x': 246.25, 'orientation_y': 83.25}, {'time': 1540331.0, 'accel_x': 0.14, 'accel_y': -0.17, 'accel_z': -0.16, 'gyro_x': -0.01, 'gyro_

the segment:  [{'time': 1542518.0, 'accel_x': -0.02, 'accel_y': -0.12, 'accel_z': 0.09, 'gyro_x': 0.0, 'gyro_y': 0.01, 'gyro_z': 0.01, 'mag_x': -44.25, 'mag_y': -2.37, 'mag_z': -31.19, 'orientation_x': 241.12, 'orientation_y': 74.62}, {'time': 1542529.0, 'accel_x': 0.0, 'accel_y': -0.1, 'accel_z': 0.08, 'gyro_x': 0.07, 'gyro_y': -0.02, 'gyro_z': 0.0, 'mag_x': -44.25, 'mag_y': -2.37, 'mag_z': -31.19, 'orientation_x': 241.12, 'orientation_y': 74.56}, {'time': 1542539.0, 'accel_x': 0.01, 'accel_y': 0.0, 'accel_z': 0.08, 'gyro_x': 0.1, 'gyro_y': -0.01, 'gyro_z': -0.01, 'mag_x': -44.0, 'mag_y': -1.56, 'mag_z': -32.75, 'orientation_x': 241.06, 'orientation_y': 74.56}, {'time': 1542549.0, 'accel_x': 0.01, 'accel_y': 0.0, 'accel_z': 0.07, 'gyro_x': 0.08, 'gyro_y': 0.01, 'gyro_z': -0.01, 'mag_x': -44.0, 'mag_y': -1.56, 'mag_z': -32.75, 'orientation_x': 241.0, 'orientation_y': 74.56}, {'time': 1542560.0, 'accel_x': 0.01, 'accel_y': -0.01, 'accel_z': 0.07, 'gyro_x': 0.03, 'gyro_y': 0.01, 'gyro_z'

the segment:  [{'time': 1544786.0, 'accel_x': 0.08, 'accel_y': -0.62, 'accel_z': -0.02, 'gyro_x': -0.21, 'gyro_y': -0.1, 'gyro_z': 0.11, 'mag_x': -44.19, 'mag_y': -2.19, 'mag_z': -31.56, 'orientation_x': 240.62, 'orientation_y': 74.37}, {'time': 1544796.0, 'accel_x': 0.03, 'accel_y': -0.55, 'accel_z': -0.27, 'gyro_x': 0.19, 'gyro_y': 0.07, 'gyro_z': 0.1, 'mag_x': -43.5, 'mag_y': -3.56, 'mag_z': -33.25, 'orientation_x': 240.44, 'orientation_y': 74.37}, {'time': 1544808.0, 'accel_x': 0.06, 'accel_y': 0.26, 'accel_z': -0.37, 'gyro_x': 0.4, 'gyro_y': 0.1, 'gyro_z': 0.04, 'mag_x': -43.5, 'mag_y': -3.56, 'mag_z': -33.25, 'orientation_x': 240.12, 'orientation_y': 74.44}, {'time': 1544820.0, 'accel_x': 0.02, 'accel_y': 0.23, 'accel_z': 0.27, 'gyro_x': 0.1, 'gyro_y': -0.02, 'gyro_z': 0.01, 'mag_x': -43.5, 'mag_y': -3.56, 'mag_z': -33.25, 'orientation_x': 239.87, 'orientation_y': 74.5}, {'time': 1544830.0, 'accel_x': 0.12, 'accel_y': -0.11, 'accel_z': 0.0, 'gyro_x': -0.08, 'gyro_y': -0.06, 'gyro

the segment:  [{'time': 1547062.0, 'accel_x': 0.25, 'accel_y': -0.24, 'accel_z': -0.33, 'gyro_x': -0.01, 'gyro_y': 0.02, 'gyro_z': -0.01, 'mag_x': -41.06, 'mag_y': -3.25, 'mag_z': -36.25, 'orientation_x': 250.12, 'orientation_y': 80.0}, {'time': 1547076.0, 'accel_x': 0.57, 'accel_y': -0.43, 'accel_z': -0.71, 'gyro_x': 0.0, 'gyro_y': 0.01, 'gyro_z': -0.02, 'mag_x': -41.06, 'mag_y': -3.25, 'mag_z': -36.25, 'orientation_x': 250.12, 'orientation_y': 80.0}, {'time': 1547086.0, 'accel_x': 0.64, 'accel_y': -0.45, 'accel_z': -0.85, 'gyro_x': 0.04, 'gyro_y': -0.01, 'gyro_z': -0.04, 'mag_x': -41.06, 'mag_y': -3.25, 'mag_z': -36.25, 'orientation_x': 250.12, 'orientation_y': 80.0}, {'time': 1547096.0, 'accel_x': 0.72, 'accel_y': -0.4, 'accel_z': -1.0, 'gyro_x': 0.08, 'gyro_y': -0.01, 'gyro_z': -0.07, 'mag_x': -41.06, 'mag_y': -4.0, 'mag_z': -36.0, 'orientation_x': 250.12, 'orientation_y': 80.0}, {'time': 1547108.0, 'accel_x': 0.51, 'accel_y': 0.0, 'accel_z': -0.77, 'gyro_x': 0.12, 'gyro_y': 0.04, 

the segment:  [{'time': 1549339.0, 'accel_x': -1.24, 'accel_y': 0.52, 'accel_z': 1.87, 'gyro_x': 0.05, 'gyro_y': 0.12, 'gyro_z': -0.08, 'mag_x': -41.5, 'mag_y': -3.19, 'mag_z': -32.69, 'orientation_x': 246.06, 'orientation_y': 78.44}, {'time': 1549349.0, 'accel_x': -1.69, 'accel_y': 0.6, 'accel_z': 2.62, 'gyro_x': -0.14, 'gyro_y': 0.11, 'gyro_z': -0.01, 'mag_x': -41.5, 'mag_y': -3.19, 'mag_z': -32.69, 'orientation_x': 246.12, 'orientation_y': 78.5}, {'time': 1549361.0, 'accel_x': -2.26, 'accel_y': 0.46, 'accel_z': 3.22, 'gyro_x': -0.36, 'gyro_y': 0.18, 'gyro_z': 0.09, 'mag_x': -41.5, 'mag_y': -3.19, 'mag_z': -32.69, 'orientation_x': 246.19, 'orientation_y': 78.56}, {'time': 1549374.0, 'accel_x': -2.69, 'accel_y': 0.35, 'accel_z': 3.78, 'gyro_x': -0.38, 'gyro_y': 0.22, 'gyro_z': 0.2, 'mag_x': -41.5, 'mag_y': -3.19, 'mag_z': -32.69, 'orientation_x': 246.56, 'orientation_y': 78.81}, {'time': 1549384.0, 'accel_x': -3.45, 'accel_y': 0.36, 'accel_z': 4.61, 'gyro_x': -0.27, 'gyro_y': 0.11, 'g

In [56]:
# arduino_proc.join()
# stride_process.join()
# strides_compute_proc.join()
arduino_proc.terminate()
stride_process.terminate()
strides_compute_proc.terminate()

In [47]:
total_strides.empty()

True

In [31]:
one_stride = total_strides.get()

KeyboardInterrupt: 

In [ ]:
one_stride

In [ ]:
df_walk = one_stride.df_walk_data
accel_z_smooth = one_stride.accel_z_smooth

In [ ]:
fig = plt.figure(figsize=(14, 4), dpi=80)

In [ ]:
plt.plot(df_walk.time,df_walk.accel_z)
plt.plot(df_walk.time,accel_z_smooth)
plt.axhline(y=upper_limit, color='r', linestyle='-')
plt.axhline(y=lower_limit, color='r', linestyle='-')


In [ ]:

while True:
    try:
        plt.plot(df_walk.time,df_walk.accel_z)
        plt.plot(df_walk.time,accel_z_smooth)
        plt.axhline(y=upper_limit, color='r', linestyle='-')
        plt.axhline(y=lower_limit, color='r', linestyle='-')
#         plt.show()
        plt.draw()
        print ("Number of Strides: ", len(strides))
        for idx, stride in enumerate(strides):
            print ("Stride ", idx, " :") 
            print ("Stride Start Time: ", df_walk.time[stride.start_idx]/1000)
            print ("Stride End Time: ", df_walk.time[stride.stop_idx]/1000)
            print ("Stride Length (m):", stride.horizontal_length)
            print ("Stride Time Duration (sec): ", stride.time_duration)
            print ("Stride Time Duration (m/s): ", stride.speed)

        time.sleep(2)
    except:
        print("exception in plotting loop")


In [ ]:
total_data.empty()

In [ ]:
total_strides.empty()

In [ ]:

                    
                    
                    # print ((df_walk.shape))
            fig = plt.figure(figsize=(14, 4), dpi=80)
            plt.plot(df_walk.time,df_walk.accel_z)
            plt.plot(df_walk.time,accel_z_smooth)
            plt.axhline(y=upper_limit, color='r', linestyle='-')
            plt.axhline(y=lower_limit, color='r', linestyle='-')
            plt.show()
            print ("Number of Strides: ", len(strides))
            for idx, stride in enumerate(strides):
                print ("Stride ", idx, " :") 
                print ("Stride Start Time: ", df_walk.time[stride.start_idx]/1000)
                print ("Stride End Time: ", df_walk.time[stride.stop_idx]/1000)
                print ("Stride Length (m):", stride.horizontal_length)
                print ("Stride Time Duration (sec): ", stride.time_duration)
                print ("Stride Time Duration (m/s): ", stride.speed)

            time.sleep(1)

In [ ]:
total_data

In [ ]:
total_data.empty()

In [ ]:
stride_1 = total_data.get()

In [ ]:
stride_1

In [ ]:
# scratch

In [ ]:
walk_segment = total_data.get()
print("the segment: ", walk_segment)
for k in walk_segment:total_walk.append(k) 
df_walk = pd.DataFrame(total_walk)
df_walk.time = df_walk.time - df_walk.time[0] # reset time from start of stream
# Filter out accel_z
accel_z_smooth = gaussian_filter1d(df_walk.accel_z, gaussian_filter_sigma)
# Define Baseline Noise

accel_z_std = np.std(df_walk.accel_z[start_quiet_noise_idx:end_quiet_noise_idx], axis=0)
accel_z_mean = np.mean(df_walk.accel_z[start_quiet_noise_idx:end_quiet_noise_idx])
upper_limit = accel_z_mean + num_of_stddev * accel_z_std
lower_limit = accel_z_mean - num_of_stddev * accel_z_std

# Determining Stride
# Stride Class with auto-compute all chracterisitcs
stride_under = False
stride_over = False
start_stride_idx, end_stride_idx = None, None


In [ ]:

# Determining Stride
# Stride Class with auto-compute all chracterisitcs
stride_under = False
stride_over = False
start_stride_idx, end_stride_idx = None, None

for idx, data in enumerate(accel_z_smooth):
    # timeout
    if start_stride_idx and idx-start_stride_idx > STRIDE_TIMEOUT:
        start_stride_idx = None
        stride_under = False
        stride_over = False
    if (data < lower_limit) and (not stride_under and not stride_over):
        # Heal off the ground
        stride_under = True
        start_stride_idx = idx
    elif (data > upper_limit) and (stride_under and not stride_over):
        stride_under = False
        stride_over = True
    elif (data < upper_limit) and (not stride_under and stride_over):
        stride_under = False
        stride_over = False   
        end_stride_idx = idx
        curr_stride = Stride (df_walk, start_stride_idx, end_stride_idx)
        strides.append(curr_stride)



In [ ]:
for idx, data in enumerate(accel_z_smooth):
    print("idx: ", idx)
    print("data: ", data)

In [ ]:
strides

In [ ]:
decoded_bytes = "time=38758,accel_x= 0.03,accel_y= 0.03,accel_z= -0.39,gyro_x= 0.00,gyro_y= -0.00,gyro_z= -0.00,mag_x= 4.50,mag_y= 3.06,mag_z= -10.50,orientation_x= 359.94,orientation_y= 4.31,orientation_z= 0.44"

In [ ]:
row = process_serial_out(decoded_bytes)
print(decoded_bytes)

In [ ]:
ascii_string = decoded_bytes
ascii_string += ','
findall = re.findall(r'(?P<var>.*?)=(?P<out>.*?),', ascii_string)
extracted = {k.strip().lower(): float(v.strip()) for k,v in findall}


In [ ]:
extracted